In [1]:
import pandas as pd
import ast
import requests
import base64

### Prepare a Clean Example File
1. product title 
2. product images >> convert into a list with 5 images, drop gif, support png, jpg, webp
3. product description 

In [2]:

# Load the data from the file into a DataFrame
file_path = '\exampleraw_examples.csv'
df_example = pd.read_csv(file_path, encoding='utf-8')
# df_example.head()

In [62]:
def combine_images_to_list(row):
    image_columns = [col for col in row.index if 'images' in col]
    images = []
    for col in image_columns:
        if pd.notna(row[col]):
            image_url = 'https:' + row[col]
            images.append(image_url)
    return images

df_example['product_images'] = df_example.apply(combine_images_to_list, axis=1)
df_example.drop(columns=['images 1', 'images 2', 'images 3', 'images 4', 'images 5'], inplace=True, errors='ignore')

In [63]:
df_example.head()
# for key, value in df_example.iloc[0].items():
#     print(f"{key}: {value}")

,address,product_title,description,product_images
0,https://www.pazzion.com/products/191-5a-fiorel...,Fiorella Boots,"A perfect blend of style, comfort, and conveni...",[https://www.pazzion.com/cdn/shop/files/Fiorel...
1,https://www.pazzion.com/products/2203a-1-mered...,Meredith Ankle Lace-up Boots,The perfect blend of style and comfort for the...,[https://www.pazzion.com/cdn/shop/files/Meredi...
2,https://www.pazzion.com/products/2597-1-ingrid...,Ingrid Gold Studded Cage Patent Leather Slides,Gladiator styled sandals made in glossy patent...,[https://www.pazzion.com/cdn/shop/files/Ingrid...
3,https://www.pazzion.com/products/522-1-kate-un...,Kate Unlace Platform Sneakers,Athletic trainers don’t need to be dedicated t...,[https://www.pazzion.com/cdn/shop/files/KateUn...
4,https://www.pazzion.com/products/80-29-reagan-...,Reagan Contrast Sneakers,A pristine clean white sneaker is essential to...,[https://www.pazzion.com/cdn/shop/files/Reagan...


In [48]:
# first_row_product_images = df_example.loc[1, 'product_images']
# print(type(first_row_product_images))
# print(first_row_product_images)

['https://www.pazzion.com/cdn/shop/files/MaiaPearlDi_corBowSlingbacksKittenHeels-Beige-1.webp?v=1705903988', 'https://www.pazzion.com/cdn/shop/files/MaiaPearlDi_corBowSlingbacksKittenHeels-Beige-2.webp?v=1705903993', 'https://www.pazzion.com/cdn/shop/files/MaiaPearlDi_corBowSlingbacksKittenHeels-Beige-3.webp?v=1705903999', 'https://www.pazzion.com/cdn/shop/files/MaiaPearlDi_corBowSlingbacksKittenHeels-Beige-4.webp?v=1705904009', 'https://www.pazzion.com/cdn/shop/files/MaiaPearlDi_corBowSlingbacksKittenHeels-Beige-5.webp?v=1705904014']


In [64]:
output_file_path = 'examples.csv'
df_example.to_csv(output_file_path, index=False, encoding='utf-8')


### Base64


In [10]:
file_path = 'claude/test_products_base64.csv'
df_example = pd.read_csv(file_path, encoding='utf-8')

In [11]:
# convert product_images from a str to a list

def convert_string_to_list(string_list):
    try:
        actual_list = ast.literal_eval(string_list)
        return actual_list
    except ValueError as e:
        print(f"Error converting string to list: {e}")
        return None

df_example['product_images'] = df_example['product_images'].apply(convert_string_to_list)


In [12]:
def get_base64_encoded_image(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        binary_data = response.content
        media_type = response.headers['Content-Type']
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode('utf-8')
        return base64_string, media_type
    else:
        print(f"Error fetching image: {response.status_code}")
        return None, None
    
df_example['product_images_base64'], df_example['media_type'] = zip(*df_example['product_images'].apply(lambda images: zip(*[get_base64_encoded_image(image) for image in images])))


In [13]:
df_example.head()

,address,product_title,product_images,product_images_base64,media_type
0,https://www.pazzion.com/products/c2257-3-joyce...,Joyce Pearl Patent Slingback Heels,[https://www.pazzion.com/cdn/shop/files/JoyceP...,(/9j/4QCuRXhpZgAASUkqAAgAAAAGABIBAwABAAAAAQAAA...,"(image/jpeg, image/jpeg, image/jpeg, image/jpe..."
1,https://www.pazzion.com/products/122-5-maia-pe...,Maia Pearl Décor Bow Slingbacks Kitten Heels,[https://www.pazzion.com/cdn/shop/files/MaiaPe...,(/9j/4QCuRXhpZgAASUkqAAgAAAAGABIBAwABAAAAAQAAA...,"(image/jpeg, image/jpeg, image/jpeg, image/jpe..."
2,https://www.pazzion.com/products/6313-1-aureli...,Aurelia Strappy Espadrilles,[https://www.pazzion.com/cdn/shop/files/Aureli...,(/9j/4QC8RXhpZgAASUkqAAgAAAAGABIBAwABAAAAAQAAA...,"(image/jpeg, image/jpeg, image/jpeg, image/jpe..."
3,https://www.pazzion.com/products/1919-1-margot...,Margot Tweed Lace-up Espadrilles,[https://www.pazzion.com/cdn/shop/files/Margot...,(/9j/4QC8RXhpZgAASUkqAAgAAAAGABIBAwABAAAAAQAAA...,"(image/jpeg, image/jpeg, image/jpeg, image/jpe..."
4,https://www.pazzion.com/products/3899-9-fallon...,Fallon Leather Ankle Boots,[https://www.pazzion.com/cdn/shop/files/Fallon...,(/9j/4QC8RXhpZgAASUkqAAgAAAAGABIBAwABAAAAAQAAA...,"(image/jpeg, image/jpeg, image/jpeg, image/jpe..."


### write a cleaned_example.csv

In [14]:
output_file_path = 'claude/test_products_base64.csv'
df_example.to_csv(output_file_path, index=False)


### preview

In [56]:
file_path = 'cleaned_examples.csv'
df = pd.read_csv(file_path, encoding='utf-8')

for key, value in df.iloc[0].items():
    print(f"{key}: {value}")

address: https://www.pazzion.com/products/191-5a-fiorella-boots
product_title: Fiorella Boots
description: A perfect blend of style, comfort, and convenience, crafted to cater to the modern woman's needs, Fiorella is designed with a focus on ease of wear and contemporary style. These boots feature a convenient zipper closure, making them a breeze to put on and take off. No more struggling with laces or buttons; whether you're dashing through your daily routine or stepping out for a night on the town, just zip up and you're ready to conquer your day.
product_images: ['https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-1.webp?v=1705915480', 'https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-10.webp?v=1715759531', 'https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-2.webp?v=1715759531', 'https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-3.webp?v=1715759531', 'https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-4.webp?v=1715759531']
product_images_b

In [60]:
print(type(df['product_images_base64'].iloc[0]))

<class 'str'>
